# Instalación

### Instalación de Hadoop

Instalamos la versión 3.3.4 de Hadoop. Se puede probar con otras versiones más recientes como en otros cuadernos, pero siempre cuidando posteriormente que las versiones del resto de componentes (sqoop, hive, pig, etc.) sean compatibles. 

In [ ]:
%%bash
wget https://downloads.apache.org/hadoop/common/hadoop-3.3.4/hadoop-3.3.4.tar.gz
tar -xzf hadoop-3.3.4.tar.gz
mv  hadoop-3.3.4/ /usr/local/

--2023-03-31 08:57:23--  https://downloads.apache.org/hadoop/common/hadoop-3.3.4/hadoop-3.3.4.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 695457782 (663M) [application/x-gzip]
Saving to: ‘hadoop-3.3.4.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  244K 46m28s
    50K .......... .......... .......... .......... ..........  0%  486K 34m52s
   100K .......... .......... .......... .......... ..........  0% 20.0M 23m26s
   150K .......... .......... .......... .......... ..........  0% 19.1M 17m43s
   200K .......... .......... .......... .......... ..........  0%  502K 18m41s
   250K .......... .......... .......... .......... ..........  0% 21.2M 15m39s
   300K .......... .......... .......... .......... ..........  0% 21.2M 13m29s
   35

Actualizamos variables de entorno

In [ ]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/hadoop-3.3.4/bin"

Comprobamos instalación

In [ ]:
!hadoop version

Hadoop 3.3.4
Source code repository https://github.com/apache/hadoop.git -r a585a73c3e02ac62350c136643a5e7f6095a3dbb
Compiled by stevel on 2022-07-29T12:32Z
Compiled with protoc 3.7.1
From source with checksum fb9dd8918a7b8a5b430d61af858f6ec
This command was run using /usr/local/hadoop-3.3.4/share/hadoop/common/hadoop-common-3.3.4.jar


# Flume

### Instalación de Flume



In [ ]:
%%bash
wget https://dlcdn.apache.org/flume/1.11.0/apache-flume-1.11.0-bin.tar.gz
tar xzf apache-flume-1.11.0-bin.tar.gz
mv apache-flume-1.11.0-bin/ /usr/local/

--2023-03-31 08:58:24--  https://dlcdn.apache.org/flume/1.11.0/apache-flume-1.11.0-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87380462 (83M) [application/x-gzip]
Saving to: ‘apache-flume-1.11.0-bin.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 17.3M 5s
    50K .......... .......... .......... .......... ..........  0% 25.5M 4s
   100K .......... .......... .......... .......... ..........  0% 64.4M 3s
   150K .......... .......... .......... .......... ..........  0% 48.2M 3s
   200K .......... .......... .......... .......... ..........  0% 63.2M 2s
   250K .......... .......... .......... .......... ..........  0% 73.4M 2s
   300K .......... .......... .......... .......... ..........  0% 68.3M 2s
   350K .......... .......... .......... .......... ..........  0% 82.2M 2s


In [ ]:
os.environ["FLUME_HOME"] = "/usr/local/apache-flume-1.11.0-bin"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/apache-flume-1.11.0-bin/bin"
os.environ["JAVA_OPTS"]= "-Xms400m -Xmx400m -Dcom.sun.management.jmxremote"

In [ ]:
!flume-ng version

Flume 1.11.0
Source code repository: https://git.apache.org/repos/asf/flume.git
Revision: 1a15927e594fd0d05a59d804b90a9c31ec93f5e1
Compiled by rgoers on Sun Oct 16 14:44:15 MST 2022
From source with checksum bbbca682177262aac3a89defde369a37


[[1]](http://flume.apache.org/) Página Web Original

[[2]](https://data-flair.training/blogs/apache-flume-installation-tutorial/) Tutorial


### Ejecución de Flume

1. Preparación del entorno



Para ejecutar un trabajo flume tenemos que crear un archivo de configuración. Aquí he creaso un fichero de configuración de ejemplo que sirve para leer datos de un fichero de texto y escribirlos en un fichero local:

In [ ]:
!hdfs dfs -mkdir flume
!hdfs dfs -mkdir output

mkdir: `flume': File exists
mkdir: `data': File exists


In [ ]:
%%writefile flume/flume.conf
# Define the source, channel, and sink
example.sources = r1
example.channels = mem1
example.sinks = k1

# Configure the source
example.sources.r1.type = netcat
example.sources.r1.bind = localhost
example.sources.r1.port = 44444

# Configure the channel
example.channels.mem1.type = memory
example.channels.mem1.capacity = 1000
example.channels.mem1.transactionCapacity = 100

# Configure the sink
example.sinks.k1.type = logger

# Bind the source and sink to the channel
example.sources.r1.channels = mem1
example.sinks.k1.channel = mem1



Overwriting flume/flume.conf


3. Ejecución del Agente

In [ ]:
!sudo apt install netcat

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  netcat-openbsd
The following NEW packages will be installed:
  netcat netcat-openbsd
0 upgraded, 2 newly installed, 0 to remove and 23 not upgraded.
Need to get 40.0 kB of archives.
After this operation, 126 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 netcat-openbsd amd64 1.206-1ubuntu1 [37.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 netcat all 1.206-1ubuntu1 [2,172 B]
Fetched 40.0 kB in 0s (109 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf:

In [ ]:
!flume-ng agent --conf-file /content/flume/flume.conf --name example -Dflume.root.logger=INFO,console -Xmx1000m 

Info: Including Hadoop libraries found via (/usr/local/hadoop-3.3.4/bin/hadoop) for HDFS access
Info: Including Hive libraries found via () for Hive access
+ exec /usr/lib/jvm/java-11-openjdk-amd64/bin/java -Xmx20m -Dflume.root.logger=INFO,console -Xmx1000m -cp '/usr/local/apache-flume-1.11.0-bin/lib/*:/usr/local/hadoop-3.3.4/etc/hadoop:/usr/local/hadoop-3.3.4/share/hadoop/common/lib/*:/usr/local/hadoop-3.3.4/share/hadoop/common/*:/usr/local/hadoop-3.3.4/share/hadoop/hdfs:/usr/local/hadoop-3.3.4/share/hadoop/hdfs/lib/*:/usr/local/hadoop-3.3.4/share/hadoop/hdfs/*:/usr/local/hadoop-3.3.4/share/hadoop/mapreduce/*:/usr/local/hadoop-3.3.4/share/hadoop/yarn:/usr/local/hadoop-3.3.4/share/hadoop/yarn/lib/*:/usr/local/hadoop-3.3.4/share/hadoop/yarn/*:/lib/*' -Djava.library.path=::/usr/java/packages/lib:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib org.apache.flume.node.Application --conf-file /content/flume/flume.conf --name example
SLF

In [ ]:
!echo "hello world" | nc localhost 44444


# Sqoop

### Instalación de Sqoop

1. Download sqoop


In [ ]:
%%bash 
wget http://archive.apache.org/dist/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
tar -xf sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
mv sqoop-1.4.7.bin__hadoop-2.6.0 /usr/local/sqoop 

--2023-03-31 09:18:46--  http://archive.apache.org/dist/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17953604 (17M) [application/x-gzip]
Saving to: ‘sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  264K 66s
    50K .......... .......... .......... .......... ..........  0%  561K 49s
   100K .......... .......... .......... .......... ..........  0% 1.12M 37s
   150K .......... .......... .......... .......... ..........  1% 1.27M 31s
   200K .......... .......... .......... .......... ..........  1% 1.25M 28s
   250K .......... .......... .......... .......... ..........  1% 1.23M 25s
   300K .......... .......... .......... .......... ..........  1% 1.28M 23s
   350K .......... .......... .....

2. Download libraries

In [ ]:
%%bash
wget https://dlcdn.apache.org//commons/lang/binaries/commons-lang-2.6-bin.tar.gz
tar -xf commons-lang-2.6-bin.tar.gz
cp /content/commons-lang-2.6/commons-lang-2.6.jar /usr/local/sqoop/lib

--2023-03-31 09:19:01--  https://dlcdn.apache.org//commons/lang/binaries/commons-lang-2.6-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3418617 (3.3M) [application/x-gzip]
Saving to: ‘commons-lang-2.6-bin.tar.gz’

     0K .......... .......... .......... .......... ..........  1% 33.3M 0s
    50K .......... .......... .......... .......... ..........  2% 38.0M 0s
   100K .......... .......... .......... .......... ..........  4% 20.6M 0s
   150K .......... .......... .......... .......... ..........  5% 53.8M 0s
   200K .......... .......... .......... .......... ..........  7% 81.4M 0s
   250K .......... .......... .......... .......... ..........  8% 60.6M 0s
   300K .......... .......... .......... .......... .......... 10% 72.7M 0s
   350K .......... .......... .......... .......... .......... 11% 58.6M

3. Variables de Entorno

In [ ]:
import os
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.4"
os.environ["HADOOP_COMMON_HOME"] = "/usr/local/hadoop-3.3.4" 
os.environ["HADOOP_MAPRED_HOME"] = "/usr/local/hadoop-3.3.4"
os.environ["SQOOP_HOME"] = "/usr/local/sqoop"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/sqoop/bin" 
os.environ["HADOOP_COMMON_LIB_NATIVE_DIR"] = "/usr/local/hadoop-3.3.4/lib/native"
os.environ["CLASSPATH"] = "/content/tmp"
# variables de partes no instaladas pero que sqoop busca (eliminar warnings)
os.environ["HBASE_HOME"] = "/usr/local/hadoop-3.3.4"
os.environ["HCAT_HOME"] ="/usr/local/hadoop-3.3.4"
os.environ["ACCUMULO_HOME"] ="/usr/local/hadoop-3.3.4"
os.environ["ZOOKEEPER_HOME"] = "/usr/local/hadoop-3.3.4"

In [ ]:
os.environ["CLASSPATH"]

'/content/tmp'

4. Test

In [ ]:
!sqoop-version

Error: /usr/local/hadoop-3.3.1 does not exist!
Please set $HADOOP_COMMON_HOME to the root of your Hadoop installation.


 Revisión de los drivers y librerías preinstaladas en sqoop

In [ ]:
!ls "/usr/local/sqoop/lib"

ant-contrib-1.0b3.jar	       kite-data-core-1.1.0.jar
ant-eclipse-1.0-jvm1.2.jar     kite-data-hive-1.1.0.jar
avro-1.8.1.jar		       kite-data-mapreduce-1.1.0.jar
avro-mapred-1.8.1-hadoop2.jar  kite-hadoop-compatibility-1.1.0.jar
commons-codec-1.4.jar	       opencsv-2.3.jar
commons-compress-1.8.1.jar     paranamer-2.7.jar
commons-io-1.4.jar	       parquet-avro-1.6.0.jar
commons-jexl-2.1.1.jar	       parquet-column-1.6.0.jar
commons-lang-2.6.jar	       parquet-common-1.6.0.jar
commons-lang3-3.4.jar	       parquet-encoding-1.6.0.jar
commons-logging-1.1.1.jar      parquet-format-2.2.0-rc1.jar
hsqldb-1.8.0.10.jar	       parquet-generator-1.6.0.jar
jackson-annotations-2.3.1.jar  parquet-hadoop-1.6.0.jar
jackson-core-2.3.1.jar	       parquet-jackson-1.6.0.jar
jackson-core-asl-1.9.13.jar    slf4j-api-1.6.1.jar
jackson-databind-2.3.1.jar     snappy-java-1.1.1.6.jar
jackson-mapper-asl-1.9.13.jar  xz-1.5.jar


Sqlite



```
gdown --id 0BwUVtTEFk6GAMmc0WmF0SUl2d3c
mv sqlite-jdbc-3.7.2.jar "/usr/local/sqoop/lib"
```



In [ ]:
%%bash
wget http://ftp.ntu.edu.tw/MySQL/Downloads/Connector-J/mysql-connector-java-5.1.49.tar.gz
tar -zxf mysql-connector-java-5.1.49.tar.gz
cd mysql-connector-java-5.1.49
mv mysql-connector-java-5.1.49-bin.jar /usr/local/sqoop/lib
ls /usr/local/sqoop/lib

ant-contrib-1.0b3.jar
ant-eclipse-1.0-jvm1.2.jar
avro-1.8.1.jar
avro-mapred-1.8.1-hadoop2.jar
commons-codec-1.4.jar
commons-compress-1.8.1.jar
commons-io-1.4.jar
commons-jexl-2.1.1.jar
commons-lang-2.6.jar
commons-lang3-3.4.jar
commons-logging-1.1.1.jar
hsqldb-1.8.0.10.jar
jackson-annotations-2.3.1.jar
jackson-core-2.3.1.jar
jackson-core-asl-1.9.13.jar
jackson-databind-2.3.1.jar
jackson-mapper-asl-1.9.13.jar
kite-data-core-1.1.0.jar
kite-data-hive-1.1.0.jar
kite-data-mapreduce-1.1.0.jar
kite-hadoop-compatibility-1.1.0.jar
mysql-connector-java-5.1.49-bin.jar
opencsv-2.3.jar
paranamer-2.7.jar
parquet-avro-1.6.0.jar
parquet-column-1.6.0.jar
parquet-common-1.6.0.jar
parquet-encoding-1.6.0.jar
parquet-format-2.2.0-rc1.jar
parquet-generator-1.6.0.jar
parquet-hadoop-1.6.0.jar
parquet-jackson-1.6.0.jar
slf4j-api-1.6.1.jar
snappy-java-1.1.1.6.jar
xz-1.5.jar


--2023-03-31 09:19:02--  http://ftp.ntu.edu.tw/MySQL/Downloads/Connector-J/mysql-connector-java-5.1.49.tar.gz
Resolving ftp.ntu.edu.tw (ftp.ntu.edu.tw)... 140.112.36.185
Connecting to ftp.ntu.edu.tw (ftp.ntu.edu.tw)|140.112.36.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3362563 (3.2M) [application/octet-stream]
Saving to: ‘mysql-connector-java-5.1.49.tar.gz’

     0K .......... .......... .......... .......... ..........  1%  141K 23s
    50K .......... .......... .......... .......... ..........  3%  141K 23s
   100K .......... .......... .......... .......... ..........  4%  281K 19s
   150K .......... .......... .......... .......... ..........  6%  281K 16s
   200K .......... .......... .......... .......... ..........  7%  282K 15s
   250K .......... .......... .......... .......... ..........  9% 57.0M 12s
   300K .......... .......... .......... .......... .......... 10%  281K 12s
   350K .......... .......... .......... .......... .......... 12%

### Ejecución de Sqoop

La base de datos Rfam es una colección de familias de secuencias de ARN estructurales que incluyen genes de ARN no codificantes, así como elementos cis-reguladores. Cada familia está representada por un alineamiento múltiple de secuencias y un modelo de covarianza (MC).

[Enlace](https://docs.rfam.org/en/latest/about-rfam.html)



```
sqoop import 
  --connect jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam 
  --username rfamro --password "" 
  --table family --m 1 
```



In [ ]:
%%bash
sqoop import -Dmapreduce.job.user.classpath.first=true \
  --connect jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam  \
  --username rfamro --password "" \
  --table family \
  --m 1 \
  --bindir /usr/local/hadoop-3.3.4/etc/hadoop \
  --class-name family \

2023-03-31 09:24:29,438 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
2023-03-31 09:24:29,495 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
2023-03-31 09:24:29,729 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
2023-03-31 09:24:29,729 INFO tool.CodeGenTool: Beginning code generation
2023-03-31 09:24:31,224 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `family` AS t LIMIT 1
2023-03-31 09:24:31,850 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `family` AS t LIMIT 1
2023-03-31 09:24:32,186 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /usr/local/hadoop-3.3.4
Note: /usr/local/hadoop-3.3.4/etc/hadoop/family.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
2023-03-31 09:24:36,915 INFO orm.CompilationManager: Writing jar file: /usr/local/hadoop-3.3.4/etc/hadoop/family.jar
2023-03-31 09:24:36,971 WARN manager.MySQLMana

In [ ]:
!hdfs dfs -cat family/* | head

2023-03-31 09:24:54,053 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
RF00001,5S_rRNA,1302,5S ribosomal RNA,Griffiths-Jones SR, Mifsud W, Gardner PP,Szymanski et al, 5S ribosomal database, PMID:11752286,38.0,38.0,37.9,5S ribosomal RNA (5S rRNA) is a component of the large ribosomal subunit in both prokaryotes and eukaryotes. In eukaryotes, it is synthesised by RNA polymerase III (the other eukaryotic rRNAs are cleaved from a 45S precursor synthesised by RNA polymerase I). In Xenopus oocytes, it has been shown that fingers 4-7 of the nine-zinc finger transcription factor TFIIIA can bind to the central region of 5S RNA. Thus, in addition to positively regulating 5S rRNA transcription, TFIIIA also stabilises 5S rRNA until it is required for transcription.,null,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 24.99 -Z 549862.597050 CM SEQDB,712,139932,0,0,Gene; rRNA;,Publish

In [ ]:
!hdfs dfs -cat ./family/*

2023-03-31 09:33:24,355 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
RF00001,5S_rRNA,1302,5S ribosomal RNA,Griffiths-Jones SR, Mifsud W, Gardner PP,Szymanski et al, 5S ribosomal database, PMID:11752286,38.0,38.0,37.9,5S ribosomal RNA (5S rRNA) is a component of the large ribosomal subunit in both prokaryotes and eukaryotes. In eukaryotes, it is synthesised by RNA polymerase III (the other eukaryotic rRNAs are cleaved from a 45S precursor synthesised by RNA polymerase I). In Xenopus oocytes, it has been shown that fingers 4-7 of the nine-zinc finger transcription factor TFIIIA can bind to the central region of 5S RNA. Thus, in addition to positively regulating 5S rRNA transcription, TFIIIA also stabilises 5S rRNA until it is required for transcription.,null,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 24.99 -Z 549862.597050 CM SEQDB,712,139932,0,0,Gene; rRNA;,Publish

**RECORDATORIO**: 
-  `-m 1` Dado que es un entorno standalone no debemos forzar el split de los datos sino al contrario, establecemos el número de maps a 1
- bindir a un directorio dentro del classpath de hadoop

# Spark-Hive


En primer lugar, tenemos que instalar y configurar Apache Spark y Hive.

In [ ]:
# Install dependencies
!wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar -xvf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

# Set environment variables
import os
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# Start Spark session
import findspark
findspark.init()

# Start Hive session
!apt-get install hive -qq > /dev/null


spark-3.3.2-bin-hadoop3/
spark-3.3.2-bin-hadoop3/LICENSE
spark-3.3.2-bin-hadoop3/NOTICE
spark-3.3.2-bin-hadoop3/R/
spark-3.3.2-bin-hadoop3/R/lib/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/DESCRIPTION
spark-3.3.2-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/Rd.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/features.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/hsearch.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/links.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/nsInfo.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/package.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/vignette.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/NAMESPACE
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/SparkR
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/SparkR.rdb
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/SparkR.rdx
spark-3.3.2-bin-hadoop3/R/lib/SparkR/doc/
spark-3.3.2-bin-hadoop3/R/lib/Spar

Después, podemos crear un marco de datos Spark y almacenarlo como una tabla Hive:

In [ ]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName("example").enableHiveSupport().getOrCreate()

# Create dataframe
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Create Hive table
df.write.mode("overwrite").saveAsTable("example_table")

# Query Hive table
query = "SELECT * FROM example_table"
result = spark.sql(query)
result.show()


+-------+---+
|   Name|Age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
|  Alice| 25|
+-------+---+



Este código crea un marco de datos Spark con algunos datos, lo guarda como una tabla Hive llamada tabla_ejemplo y lo consulta utilizando SQL.